In [1]:
import pandas as pd
import os
import yaml
import numpy as np
import json
from itertools import combinations
import pretty_midi
import librosa
import matplotlib.pyplot as plt

In [2]:
# Load JSON w/ chords
json_path = './data/jsb-chorales-8th.json'
with open(json_path, "r") as f:
     chorale_data = json.loads(f.read())
print(len(chorale_data['train']),len(chorale_data['test']),len(chorale_data['valid']))

229 77 76


In [3]:
print(len(chorale_data['train'][0]), chorale_data['train'][0])

96 [[74, 70, 65, 58], [74, 70, 65, 58], [75, 70, 58, 55], [75, 70, 60, 55], [77, 69, 62, 50], [77, 69, 62, 50], [77, 70, 62, 55], [77, 69, 62, 55], [75, 67, 63, 48], [75, 69, 63, 48], [74, 70, 65, 46], [74, 70, 65, 46], [72, 69, 65, 53], [72, 69, 65, 53], [72, 69, 65, 53], [72, 69, 65, 53], [74, 70, 65, 46], [74, 70, 65, 46], [75, 69, 63, 48], [75, 67, 63, 48], [77, 65, 62, 50], [77, 65, 60, 50], [74, 67, 58, 55], [74, 67, 58, 53], [72, 67, 58, 51], [72, 67, 58, 51], [72, 65, 57, 53], [72, 65, 57, 53], [70, 65, 62, 46], [70, 65, 62, 46], [70, 65, 62, 46], [70, 65, 62, 46], [72, 69, 65, 53], [72, 69, 65, 53], [74, 71, 53, 50], [74, 71, 53, 50], [75, 72, 55, 48], [75, 72, 55, 50], [75, 67, 60, 51], [75, 67, 60, 53], [74, 67, 60, 55], [74, 67, 57, 55], [74, 65, 59, 43], [72, 63, 59, 43], [72, 63, 55, 48], [72, 63, 55, 48], [72, 63, 55, 48], [72, 63, 55, 48], [75, 67, 60, 60], [75, 67, 60, 60], [77, 70, 62, 58], [77, 70, 62, 56], [79, 70, 62, 55], [79, 70, 62, 53], [79, 70, 63, 51], [79, 7

In [4]:
pitch_to_name = {
    0:'C',
    1:'C#/Db',
    2:'D',
    3:'D#/Eb',
    4:'E',
    5:'F',
    6:'F#/Gb',
    7:'G',
    8:'G#/Ab',
    9:'A',
    10:'A#/Bb',
    11:'B'
}
def note_name_from_number(pitch):
    octave = (pitch//12) - 1
    pitch = pitch%12
    return pitch_to_name[pitch] + str (octave)

# Test
print(note_name_from_number(0), note_name_from_number(122))

C-1 D9


In [ ]:
'''
1: [0,4,7], # C
2: [2,5,9], # d
3: [4,7,11], # e
4: [5,9,0], # F
5: [7,11,2], # G
6: [9,0,4], # a
7: [11,2,5], # b
'''

def seventh_name_from_notes(notes):
    notes.sort()
    # four cases:
    #   - case #1: all in order, not over the gap (i.e., root first), e.g. C major 0 4 7, C minor 0 3 7
    #              intervals would be: 4, 3 OR 3, 4 
    #   - case #2: root is 2nd, 7th wrapped around e.g. F major F A C
    #              intervals would be: 5, 4 or 5, 3 
    #   - case #3: root is 3rd, 5th and 7th wrapped around e.g. A minor A C E
    #              intervals would be: 4, 5 or 3, 5
    #   - case #4: root is 4th, 3rd, 5th, and 7th wrapped around e.g. A minor A C E
    #              intervals would be: 4, 5 or 3, 5



def triad_name_from_notes(pitches): # assume sorted! 0-12
    notes=[]
    for pitch in pitches: 
        note = pitch%12
        if note not in notes: 
            notes.append(note)
    notes.sort()
    if len(notes) != 3:
        print(len(notes))
        return "7th"
    else: 
    # three cases:
    #   - case #1: all in order, not over the gap (i.e., root first), e.g. C major 0 4 7, C minor 0 3 7
    #              intervals would be: 4, 3 OR 3, 4 
    #                                  3, 3    4, 4
    #   - case #2: root is 2nd, 5th wrapped around e.g. F major F A C
    #              intervals would be: 5, 4 or 5, 3 
    #                                          6, 3
    #   - case #3: root is 3rd, 3rd and 5th wrapped around e.g. A minor A C E
    #              intervals would be: 4, 5 or 3, 5
    #                                  4, 4    3, 6 (dim)
    #print(pitch_to_name[notes[0]],pitch_to_name[notes[1]],pitch_to_name[notes[2]])
        interval_1 = notes[1] - notes[0]
        interval_2 = notes[2] - notes[1]
        if [interval_1, interval_2] == [4,3]:
            return pitch_to_name[notes[0]] + "maj"
        elif [interval_1, interval_2] == [3,4]:
            return pitch_to_name[notes[0]] + "min"
        elif [interval_1, interval_2] == [4,4]:
            return pitch_to_name[notes[0]] + "aug"
        elif [interval_1, interval_2] == [3,3]:
            return pitch_to_name[notes[0]] + "dim"
        elif [interval_1, interval_2] == [5,4]:
            return pitch_to_name[notes[1]] + "maj"
        elif [interval_1, interval_2] == [5,3]:
            return pitch_to_name[notes[1]] + "min"
        elif [interval_1, interval_2] == [6,3]:
            return pitch_to_name[notes[1]] + "dim"
        elif [interval_1, interval_2] == [4,5]:
            return pitch_to_name[notes[2]] + "min"
        elif [interval_1, interval_2] == [3,6]:
            return pitch_to_name[notes[2]] + "dim"
        else: # [3,5]
            return pitch_to_name[notes[2]] + "maj"

def print_notes(chorale):
    for chord in chorale: 
        notes = ""
        for note in chord: 
            notes += note_name_from_number(note) + " "
        print(notes)

def print_chords(chorale):
    for chord in chorale: 
        print(triad_name_from_notes(chord))


print(triad_name_from_notes([0,4,7]))
print(triad_name_from_notes([1,5,8]))
print(triad_name_from_notes([2,5,9]))
print(triad_name_from_notes([4,7,11]))
print(triad_name_from_notes([9,0,4]))
print(triad_name_from_notes([5,9,0]))
print(triad_name_from_notes([11,2,5]))


#print_notes(chorale_data['train'][3])
print_chords(chorale_data['train'][3])

'''for chorale in chorale_data['train']:
    print(len(chorale))
    print_notes(chorale)
    break'''


In [ ]:
SOUNDFONT_PATH = "./GeneralUser_GS_v1.471.sf2"
import matplotlib.pyplot as plt

# synthesize a MIDI file
def midi_to_wav(midi_path,wav_path):
    cmd = "fluidsynth -F " + wav_path + ' ' + SOUNDFONT_PATH + ' ' + midi_path + ' -r 16000 -i'
    ret_status = os.system(cmd)
    if ret_status != 0:
        sys.exit(ret_status)

def chorale_to_MIDI(chorale, synth=True, path='chorale'):
    midi_obj = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=1)
    cur_time=0
    for i in range(len(chorale)): 
        for note in chorale[i]: 
            cur_note = pretty_midi.Note(velocity=100, pitch=note, start=cur_time, end=cur_time+0.5)
            instrument.notes.append(cur_note)
        cur_time += 0.5
    midi_obj.instruments.append(instrument)
    midi_obj.write(path + '.mid')
    if synth: 
        midi_to_wav(path + '.mid', path + '.wav')

def chorale_to_MIDI_2(chorale, synth=True, plot=True, path='chorale'):
    midi_obj = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=1)
    length, cur_time = 0, 0
    prev_chord = set(chorale[0])
    for i in range(len(chorale)): 
        cur_chord = set(chorale[i])
        if cur_chord != prev_chord or (i == len(chorale) - 1): # final chord or chord switch
            # create notes: 
            for note in chorale[i-1]: 
                cur_note = pretty_midi.Note(velocity=100, pitch=int(note), start=cur_time, end=cur_time+0.5*length)
                instrument.notes.append(cur_note)
            cur_time += 0.5*length
            prev_chord = cur_chord # change chord
            length = 1
        else: # cur_chord == prev_chord: 
            length += 1 #print("chord held")
    midi_obj.instruments.append(instrument)
    midi_obj.write(path + '.mid')
    if synth: 
        midi_to_wav(path + '.mid', path + '.wav')
    if plot: 
        plot_piano_roll(path+'.png', pm=midi_obj)

def plot_piano_roll(fpath, pm=None, pr=None, start_pitch=0, end_pitch=127, fs=120):
    if pr is None: 
        pr = pm.get_piano_roll(fs)#[start_pitch:end_pitch]
    print(pr)
    # Use librosa's specshow function for displaying the piano roll
    librosa.display.specshow(pr,
                             hop_length=1, sr=fs, x_axis='time', y_axis='cqt_note',
                             fmin=pretty_midi.note_number_to_hz(start_pitch))
    plt.savefig(fpath)
    plt.clf()

#chorale_to_MIDI(chorale_data['train'][2], path='old_func2')
#chorale_to_MIDI_2(chorale_data['train'][2], path='new_func2')

for i, chorale in enumerate(chorale_data['train']):
    chorale_to_MIDI_2(chorale, path='./data/chorales_synthesized/train/chorale_'+str(i), synth=False)

for i, chorale in enumerate(chorale_data['test']):
    chorale_to_MIDI_2(chorale, path='./data/chorales_synthesized/test/chorale_'+str(i), synth=False)

for i, chorale in enumerate(chorale_data['valid']):
    chorale_to_MIDI_2(chorale, path='./data/chorales_synthesized/valid/chorale_'+str(i), synth=False)


In [ ]:
notes_in_keys = {
    #        C D E F G A B
    "C/a":    [0,2,4,5,7,9,11], # C!
    "G/e":    [0,2,4,6,7,9,11], #  F#
    "D/b":    [1,2,4,6,7,9,11], # +C#
    "A/f#":    [1,2,4,6,8,9,11], # +G#
    "E/c#":    [1,3,4,6,8,9,11], # +D#
    "B-Cb/g#": [1,3,4,6,8,10,11], # +A#
    "Gb-F#/eb":[1,3,5,6,8,10,11], # +E#
    "Db-C#/bb":[0,1,3,5,6,8,10], # +B#
    "Ab/f":   [0,1,3,5,7,8,10], # +Db
    "Eb/c":   [0,2,3,5,7,8,10], # +Ab
    "Bb/g":   [0,2,3,5,7,9,10], # +Eb 
    "F/d":    [0,2,4,5,7,9,10]  # +Bb
}

maj_min_tonics = {
    #        C D E F G A B
    "C/a":  {'maj': [0,4,7], # CEG
             'min': [0,4,9]}, # ACE
    "G/e":  {'maj': [2,7,11], # GBD
             'min': [4,7,11]},  # EGB
    "D/b":  {'maj': [2,6,9], # DF#A 
             'min': [2,6,11]},  # +C#
    "A/f#":   {'maj': [1,4,9], 
               'min': [1,6,9]}, # +G#
    "E/c#":    {'maj': [4,8,11], # EG#B 
               'min': [1,4,8]}, # +D#
    "B-Cb/g#": {'maj': [3,6,11], 
               'min': [3,8,11]}, # +A#
    "Gb-F#/eb":{'maj': [1,6,10], 
               'min': [3,6,10]}, # +E#
    "Db-C#/bb":{'maj': [1,5,8], # C# F G#
               'min': [1,5,10]}, # +B#
    "Ab/f":   {'maj': [0,3,8], # G# C D#
               'min': [0,5,8]} , # +Db
    "Eb/c":   {'maj': [3,7,10], # D#GA#
               'min': [0,3,7]} , # +Ab
    "Bb/g":   {'maj': [2,5,10], # D F Bb 
               'min': [2,7,10]},# +Eb 
    "F/d":    {'maj': [0,5,9], # FAC 
               'min': [2,5,9]}   # +Bb
}

key_shift_amount = { # SUBTRACT!!!!
    "C/a":      0, # C!
    "G/e":      7, #  F#
    "D/b":      2, # +C#
    "A/f#":     9, # +G#
    "E/c#":     4, # +D#
    "B-Cb/g#":  11, # +A#
    "Gb-F#/eb": 6, # +E#
    "Db-C#/bb": 1, # +B# 
    "Ab/f":     8, # +Db
    "Eb/c":     3, # +Ab
    "Bb/g":     10, # +Eb 
    "F/d":      5   # +Bb
}

notes_in_keys_minor = {
    #        C D E F G A B
    "a":    [0,2,4,5,7,8,9,11], # C!, raised 7th is G# (8)
    "e":    [0,2,3,4,6,7,9,11], #  F# raised 7th is D# (3)
    "b":    [1,2,4,6,7,9,10,11], # +C# raised 7th: A# 10
    "f#":   [1,2,4,5,6,8,9,11], # +G# raised 7th: F 5
    "c#":   [0,1,3,4,6,8,9,11], # +D#
    "g#":   [1,3,4,5,6,8,10,11], # +A#
    "eb":   [1,2,3,5,6,8,10,11], # +E#
    "bb":   [0,1,3,5,6,8,9,10], # +B#
    "f":    [0,1,3,4,5,7,8,10], # +Db
    "c":    [0,2,3,5,7,8,10,11], # +Ab
    "g":    [0,2,3,5,6,7,9,10], # +Eb 
    "d":    [0,1,2,4,5,7,9,10]  # +Bb
}

accidental_list = [1,3,6,8,10]
flats_in_order = [10,3,8,1,6,11,4]
sharps_in_order = [6,1,8,3,10,5,0]
accidental_dict = {
    0:0,
    1:1, # 1 = is accidental 
    2:0,
    3:1,
    4:0,
    5:0,
    6:1,
    7:0,
    8:1,
    9:0,
    10:1,
    11:0
}

key_by_number_flats = {
    0:'C',
    1:'F',
    2:'Bb',
    3:'Eb',
    4:'Ab',
    5:'Db',
    6:'Gb',
}

key_by_number_sharps = {
    0:'C',
    1:'G',
    2:'D',
    3:'A',
    4:'E',
    5:'B',
    6:'F#',
}

def mode_estimate(chorale, key_estimate):
    maj_tonic = set(maj_min_tonics[key_estimate]['maj'])
    min_tonic = set(maj_min_tonics[key_estimate]['min'])
    maj_tonic_count=0
    min_tonic_count=0
    for chord in chorale: 
        notes= set()
        for pitch in chord: 
            note = pitch%12
            if note not in notes: 
                notes.add(note)
        if notes == maj_tonic:
            maj_tonic_count += 1
        if notes == min_tonic:
            min_tonic_count += 1
    if maj_tonic_count > min_tonic_count: 
        return "maj"
    else: 
        return "min"
        
def key_estimate(chorale):
    # Chorale is a list of lists of 4 pitches representing chords
    note_counts = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0,10:0,11:0}
    total_notes = len(chorale) * 4
    for chord in chorale: # count # of occurences of every pitch in the chorale, then sort most to least frequent
        for note in chord: 
            cur_note = note%12
            note_counts[cur_note] +=1
    sorted_notecounts = {k: v for k, v in sorted(note_counts.items(), key=lambda item: item[1], reverse=True)}

    #print(sorted_notecounts)
    # KEY_SUMS
    max_proportion = 0
    key_estimate = None
    for key in notes_in_keys.keys():
        key_sum = 0 
        for note in notes_in_keys[key]:
            key_sum += sorted_notecounts[note]
        key_proportion = key_sum/total_notes
        #print(key, key_sum/total_notes)
        if key_proportion > max_proportion: 
            max_proportion = key_proportion
            key_estimate = key
    #print(max_proportion, key_estimate)
    return key_estimate

train_keys = {}
for i, chorale in enumerate(chorale_data['train']):
    #print(i, triad_name_from_notes(chorale[0]),triad_name_from_notes(chorale[-1]))
    key = key_estimate(chorale)
    train_keys[int(i)]={}
    train_keys[int(i)]["key"] = key
    train_keys[int(i)]["mode"] = mode_estimate(chorale, key)

test_keys = {}
for i, chorale in enumerate(chorale_data['test']):
    #print(i, triad_name_from_notes(chorale[0]),triad_name_from_notes(chorale[-1]))
    key = key_estimate(chorale)
    test_keys[i]={}
    test_keys[i]["key"] = key
    test_keys[i]["mode"] = mode_estimate(chorale, key)

valid_keys = {}
for i, chorale in enumerate(chorale_data['valid']):
    #print(i, triad_name_from_notes(chorale[0]),triad_name_from_notes(chorale[-1]))
    key = key_estimate(chorale)
    valid_keys[i]={}
    valid_keys[i]["key"] = key
    valid_keys[i]["mode"] = mode_estimate(chorale, key)

print(train_keys)


In [60]:
import yaml 

keys_and_modes = {
    "train": train_keys,
    "test": test_keys,
    "valid": valid_keys
}

with open('./data/jsb_chorales_keys_modes.yaml', 'w') as outfile:
    yaml.dump(keys_and_modes, outfile, default_flow_style=False)


In [69]:
# Convert entire dataset to key of C/a 
chorales_in_c = {
    'train':[],
    'test':[],
    'valid':[]
}

for i, chorale in enumerate(chorale_data['train']):
    key = keys_and_modes['train'][i]['key']
    subtract_amount = key_shift_amount[key]
    new_chorale = [] 
    for chord in chorale: 
        new_chord = [] 
        for note in chord: 
            new_chord.append(int(note-subtract_amount))
        new_chorale.append(new_chord)
    chorales_in_c['train'].append(new_chorale)

for i, chorale in enumerate(chorale_data['test']):
    key = keys_and_modes['test'][i]['key']
    print(key)
    subtract_amount = key_shift_amount[key]
    new_chorale = [] 
    for chord in chorale: 
        new_chord = [] 
        for note in chord: 
            new_chord.append(int(note-subtract_amount))
        new_chorale.append(new_chord)
    chorales_in_c['test'].append(new_chorale)

for i, chorale in enumerate(chorale_data['valid']):
    key = keys_and_modes['valid'][i]['key']
    print(key)
    subtract_amount = key_shift_amount[key]
    new_chorale = [] 
    for chord in chorale: 
        new_chord = [] 
        for note in chord: 
            new_chord.append(int(note-subtract_amount))
        new_chorale.append(new_chord)
    chorales_in_c['valid'].append(new_chorale)

with open('./data/chorales_in_c.yaml', 'w') as outfile:
    yaml.dump(chorales_in_c, outfile, default_flow_style=False)

F/d
A/f#
A/f#
Bb/g
Eb/c
Bb/g
D/b
D/b
Db-C#/bb
A/f#
D/b
D/b
C/a
F/d
C/a
F/d
A/f#
Ab/f
A/f#
D/b
A/f#
C/a
E/c#
E/c#
D/b
C/a
F/d
A/f#
C/a
Bb/g
C/a
D/b
D/b
F/d
D/b
A/f#
Bb/g
Bb/g
G/e
F/d
C/a
E/c#
Bb/g
G/e
G/e
A/f#
Bb/g
A/f#
F/d
C/a
A/f#
C/a
C/a
Eb/c
D/b
C/a
D/b
C/a
G/e
G/e
G/e
D/b
Bb/g
C/a
F/d
A/f#
Bb/g
Eb/c
C/a
C/a
F/d
Eb/c
C/a
D/b
Bb/g
A/f#
Eb/c
C/a
C/a
C/a
Eb/c
F/d
D/b
Bb/g
D/b
C/a
C/a
A/f#
Bb/g
C/a
D/b
D/b
G/e
D/b
C/a
A/f#
G/e
Bb/g
G/e
A/f#
G/e
Bb/g
G/e
Bb/g
D/b
Bb/g
Bb/g
F/d
C/a
A/f#
G/e
D/b
G/e
D/b
C/a
G/e
Eb/c
C/a
F/d
A/f#
D/b
D/b
Eb/c
Bb/g
C/a
A/f#
D/b
E/c#
Db-C#/bb
Bb/g
G/e
G/e
C/a
D/b
G/e
C/a
F/d
D/b
G/e
G/e
C/a
G/e
D/b
G/e
G/e
Bb/g
G/e
F/d
C/a
Bb/g
D/b
A/f#
G/e


In [9]:
single_training_chorale = chorale_data['train'][0]
print(len(single_training_chorale))

96


In [68]:
for notes in chorale_data['train'][1]:
    print(notes)

[69, 64, 61, 57]
[69, 64, 61, 57]
[71, 64, 59, 56]
[71, 64, 57, 54]
[71, 64, 56, 52]
[71, 64, 56, 50]
[69, 64, 57, 49]
[69, 64, 57, 49]
[66, 62, 57, 50]
[66, 61, 57, 50]
[64, 59, 56, 50]
[64, 59, 56, 50]
[66, 61, 57, 49]
[68, 62, 59, 47]
[69, 64, 52, 49]
[69, 64, 57, 45]
[71, 64, 57, 52]
[71, 64, 56, 52]
[69, 64, 57, 45]
[69, 64, 57, 45]
[69, 64, 57, 45]
[69, 64, 57, 45]
[69, 64, 57, 45]
[69, 64, 57, 45]
[73, 69, 64, 57]
[74, 69, 64, 57]
[76, 71, 64, 56]
[76, 73, 64, 56]
[78, 74, 57, 54]
[78, 73, 57, 54]
[76, 71, 59, 56]
[76, 71, 59, 56]
[73, 69, 61, 57]
[73, 69, 62, 57]
[69, 69, 64, 49]
[69, 69, 64, 49]
[66, 62, 57, 50]
[68, 62, 57, 52]
[69, 61, 57, 54]
[69, 61, 57, 57]
[71, 66, 57, 50]
[71, 64, 56, 52]
[69, 61, 52, 45]
[69, 61, 52, 45]
[69, 61, 52, 45]
[69, 61, 52, 45]
[73, 69, 64, 57]
[73, 69, 64, 56]
[73, 71, 68, 54]
[73, 71, 68, 53]
[73, 69, 66, 54]
[73, 68, 66, 52]
[71, 66, 54, 50]
[71, 66, 56, 50]
[73, 66, 57, 49]
[73, 66, 57, 49]
[73, 65, 56, 49]
[73, 65, 56, 49]
[69, 61, 54, 5

In [97]:
major_to_minor_key = {
    'C':'a',
    'G':'e',
    'D':'b',
    'A':'f#',
    'E':'c#',
    'B/Cb':'ab/g#',
    'Gb/F#':'eb/d#',
    'Db/C#':'bb/a#',
    'F':'d',
    'Bb':'g',
    'Eb':'c',
    'Ab':'f',
    'Gb':'eb',
}

tonic_chords_for_keys = {
    'C':[0,4,7], # ceg
    'a':[9,0,4], # ace
    'G':[7,11,2], # gbd
    'e':[4,7,11],
    'D':[2,6,9],
    'b':[11,2,6],
    'A':[9,1,4],
    'f#':[6,10,1],
    'E':[4,8,11],
    'c#':[1,4,8],
    'B/Cb':[11,3,6],
    'ab/g#':[8,11,3],
    'Gb/F#':[6,10,1],
    'eb/d#':[3,6,10],
    'Db/C#':[0,4,7],
    'bb/a#':[10,1,5],
    'F':[5,9,0],
    'd':[2,5,9], # dfa
    'Bb':[10,2,5],
    'g':[7,10,2],
    'Eb':[3,7,10],
    'c':[0,3,7],
    'Ab':[8,0,3],
    'f':[5,8,0],
}

notes_in_keys = {
    #        C D E F G A B
    "C":    [0,2,4,5,7,9,11], # 7 notes in the key of c!
    "G":    [0,2,4,6,7,9,11], #  F#
    "D":    [1,2,4,6,7,9,11], # +C#
    "A":    [1,2,4,6,8,9,11], # +G#
    "E":    [1,3,4,6,8,9,11], # +D#
    "B/Cb": [1,3,4,6,8,10,11], # +A#
    "Gb/F#":[1,3,5,6,8,10,11], # +E#
    "Db/C#":[0,1,3,5,6,8,10], # +B#
    "Ab":   [0,1,3,5,7,8,10], # +Db
    "Eb":   [0,2,3,5,7,8,10], # +Ab
    "Bb":   [0,2,3,5,7,9,10], # +Eb 
    "F":    [0,2,4,5,7,9,10]  # +Bb
}

raised_7th_for_minor_keys = {
    #       [0,2,4,5,7,9,11]
    #        C D E F G A B
    "C":    8, # a minor, raised 7th is G-->G#
    "G":    3, #  e minor, raised 7th is D-->D#
    "D":    10, # Bm, raised 7th is A-->A#
    "A":    5, # f# minor, raised 7th is E-->F
    "E":    0, # c# minor, raised 7th is B-->C
    "B/Cb": 7, # g# minor, raised 7th is F#-->G
    "Gb/F#":2, # d# minor, raised 7th is C#-->D
    "Db/C#":9, # bb minor, raised 7th is Ab --> A
    "Ab":   4, # f minor, raised 7th is Eb --> E
    "Eb":   11, # c minor, raised 7th is Bb --> B
    "Bb":   6, # g minor, raised 7th is F-->F#
    "F":    1,  # d minor, raised 7th is C-->C#
}

def estimate_key(chorale):
#chorale = chorale data['train'][1]
    mode = "maj"
    note_counts = [0,0,0,0,0,0,0,0,0,0,0,0]
    for chord in chorale: 
        for note in chord: 
            note_counts[int(note)%12] += 1
    chosen_key = "C"
    max_key_count = 0
    max_key_count_raised = 0
    for key in notes_in_keys.keys():
        key_count = 0
        for note in notes_in_keys[key]:
            key_count += note_counts[note]
        key_count_with_raised_7th = key_count + note_counts[raised_7th_for_minor_keys[key]]
        if key_count > max_key_count:
            #print("NEW MAX", key, key_count)
            chosen_key = key
            max_key_count = key_count
            max_key_count_raised = key_count_with_raised_7th
    #print("Key:", chosen_key, max_key_count_raised/sum(note_counts) - max_key_count/sum(note_counts))
    if max_key_count_raised/sum(note_counts) - max_key_count/sum(note_counts) > 0.0137:
        key = major_to_minor_key[chosen_key]
        mode = "min"
    return chosen_key, mode

mins = 0
print("NUMBER OF CHORALES:", len(chorale_data['train']))
for chorale in chorale_data['train']:
    key, mode = estimate_key(chorale)
    if mode == 'min':
        mins += 1
print("TOTAL MINS:", mins) # should be 121


NUMBER OF CHORALES: 229
TOTAL MINS: 121


In [32]:
sorted_notecounts = key_estimate(chorale_data['train'][0])
# 3 = D#/Eb, 10=A#Bb #FCGDAEB #BEADGCF FLATS
# SO, BE, TWO FLATS!

{5: 80, 10: 77, 3: 50, 2: 49, 7: 49, 0: 48, 9: 24, 11: 4, 8: 3} 
 [5, 10, 3, 2, 7, 0, 9]


NameError: name 'x' is not defined